# Типизация в Python

## Коротко, что такое аннотации

Аннотации -- это способ указать ожидаемые типы переменных.

Например:

In [1]:
a: int


class Foo: ...


def f(x: float, y: Foo) -> None: ...

Аннотации хранятся в dunder атрибуте функции:

In [2]:
f.__annotations__

{'x': float, 'y': __main__.Foo, 'return': None}

Аннотации просто считываются и хранятся. Больше никак не обрабатываются.

Но они доступны для использования, например, в статических анализаторах кода.

## Зачем вообще аннотации типов / `mypy` (статический анализатор), если Python -- язык динамический?

Из удобства при быстром написании чего-либо, отсутствие статической типизации превращается в кошмар, например, в случаях:
- большого проекта, над которым работает множество разработчиков
- пакета с публичным API

Пример кода (где ничего не понятно):
```python
class Resource:
    ...
    def read_metadata(self, items):
        ...
```

Какие могут возникнуть вопросы у человека, которому нужно понять, что делает `read_metadata`:
- Что возвращает метод? Может ли метод вернуть None?
- Что такое `items`? Может наш любимый `list`? А если `tuple` или `set`? Или вообще `Iterable`?

Пример кода где уже яснее (благодаря аннотациям типов):
```python
class Resource:
    ...
    def read_metadata(self, items: Sequence[str]) -> Dict[str, MetadataItem]:
        ...
```

Вопросы:
- А исключения могут быть какие-то?

Пример кода где почти хорошо:
```python
class Resource:
    ...
    def read_metadata(self, items: Sequence[str]) -> Dict[str, MetadataItem]:
        """Reads `items` metadata from remote server.

        Args:
            items: The first parameter.

        Returns:
            uuid -> MetedataItem
        
        Raises:
            NotFoundError: If some item in `items` doesn't exists.
            ConnectionError: If connection to remote server isn't available.
        """
        ...
```

А "почти", потому что:
- Документацию нужно поддерживать, соответственно она может быть устаревшей
- Документация -- это не код, она может быть неточной

Стиль написание докстрингов: https://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html

<img src="kafka.jpg" style="width: 500px">
<b>Franz Kafka (c)</b>

## Чем проверять аннотации типов?

<b>Собственно, type checker, такой как <code>mypy</code> поможет вам не забыть аннотировать все вокруг типами и проверит, что вы их используете правильно.</b>

https://github.com/python/mypy

## Рантайм история -- `collections.abc`

https://docs.python.org/3/library/collections.abc.html

Этот модуль помогает понять во время исполнения программы, реализует ли класс какой-то определенный интерфейс.

In [3]:
class Foo:
    def __iter__(self): ...
    def __next__(self): ...

In [4]:
from collections.abc import Iterator, Iterable

In [5]:
assert issubclass(Foo, Iterator)
assert issubclass(Foo, Iterable)

In [6]:
issubclass(Foo(), Iterable)

TypeError: issubclass() arg 1 must be a class

In [7]:
isinstance(Foo, Iterable)

False

In [8]:
assert isinstance(Foo(), Iterable)
assert isinstance(Foo(), Iterator)

```python
class Iterable(metaclass=ABCMeta):

    ...

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterable:
            return _check_methods(C, "__iter__")
        return NotImplemented
```

Список абстрактных классов: https://docs.python.org/3/library/collections.abc.html#collections-abstract-base-classes

Просмотрите его, разберитесь, каким интерфейсам (класс может соответствовать нескольким интерфейсам сразу) соответствуют `list`, `tuple`, `dict`, `set` и почему.

## Статическая история -- `typing` и аннотации

https://docs.python.org/3/library/typing.html

Разберитесь со следующими аннотациями:
- `Any`
- `Literal["lit1", "lit2"]`
- `Union[T1, T2]`
- `Optional[T]` (то же, что и `Union[T, None]`)
- `List[T]`
- `Tuple[T, ...]`
- `Dict[KT, VT]`
- `Callable[[Arg1Type, Arg2Type], ReturnType]`

###  Как обозначать тип `*args` и `**kwargs`?

Только типы аргументов.

```python
def func(*args: int, **kwargs: float):
    ...
```

### Алиасы

```python
@contextmanager
def raises(expected: Union[Type[Exception], Tuple[Type[Exception], ...]]) -> Iterator[None]:
    try:
        yield
    except expected:
        pass
    else:
        raise Exception(f"DID NOT RAISE {e!r}")
```
__VS.__
```python
ExceptionType = Union[Type[Exception], Tuple[Type[Exception]]]  # не самое удачное именование

@contextmanager
def raises(expected: ExceptionType) -> Iterator[None]:
    try:
        yield
    except expected:
        pass
    else:
        raise Exception(f"DID NOT RAISE {e!r}")
```

### `collections.abc`

Обратите внимание, что рантаймовая и статическая истории тесно переплетаются. Модуль `typing` предоставляет ровно те же типы https://docs.python.org/3/library/typing.html#abstract-base-classes, что и `collections.abc`.

### `Protocol`

```python
class Proto(Protocol):
    def meth(self) -> int:
        ...

...

class C:
    def meth(self) -> int:
        return 0

def func(x: Proto) -> int:
    return x.meth()

func(C())  # Passes static type check
```

In [9]:
from typing import Protocol, runtime_checkable

In [10]:
@runtime_checkable
class Closable(Protocol):
    def close(self): ...

        
with open("./kafka.jpg") as f:
    assert isinstance(f, Closable)

### `Generic` / `TypeVar`

Просмотрите и проверьте с помощью `mypy` код из файла './logged_var.py'. 

Еще один пример использования `Generic` / `TypeVar`:

```python
class Mapping(Generic[KT, VT]):
    def __getitem__(self, key: KT) -> VT:
        ...
        # Etc.
```

### `self` и `cls`

Обычно не нужно аннотировать (кроме того, это может быть сложно, т.к. класс при определении методов еще не определен).

Просмотрите два примера:
- './animal_bad.py'
- './animal_good.py'

Попробуйте проанализировать их с помощью `mypy`.